# A simple demo of zeroshot
This demo reads an image from a URL, and runs a zeroshot model on it.

In [ ]:
# Install dependencies.
!pip install zeroshot
!pip install numpy requests Pillow matplotlib

In [ ]:
import io
import requests
import numpy as np
from PIL import Image

## Image loading

First, let's define a simple function to load an image from a URL.

In [ ]:
def load_image(url: str) -> np.ndarray:
    # Download the image (we'll add some headers since not all websites like the default Python user agent)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
    }
    response = requests.get(url, headers=headers, timeout=1)
    data = io.BytesIO(response.content)

    # Load and convert to Numpy array.
    img_pil = Image.open(data)
    img = np.array(img_pil)

    # Remove the alpha channel if there is one.
    if img.shape[-1] == 4:
        img = img[:, :, :3]

    # BGR to RGB
    return img


# Setup
We'll use an existing model trained to distinguish dog, cat, human, and other random zoo animals.

In [ ]:
dog_url = "https://raw.githubusercontent.com/srirammanikumar/DogBreedClassifier/master/images/Labrador_retriever_06457.jpg"
model_id = "0bf0e780-3d4c-446b-93a0-3bbea8c2b273"

image = load_image(dog_url)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5, 5))
plt.imshow(image)
plt.axis('off')
plt.show()

## Run the model
Next, just run the model! You'll need to go build a model at usezeroshot.com, and you can either download the file or just point at the GUID we provide. Here we'll just use an existing model that classifies a few existing objects.

In [ ]:
import zeroshot

# Create the classifier and preprocessing function.
classifier = zeroshot.Classifier(model_id)
preprocess_fn = zeroshot.create_preprocess_fn()

# Run the model!
prediction = classifier.predict(preprocess_fn(image))
print(f"The image is class \"{prediction}\"")

## Advanced: Show probabilities of the model
At the most basic, the model will return the most likely class. However, we can ask for probabilities to instead see how confident the model is on the various classes.

In [ ]:
# Get predictions.
prediction = classifier.predict_proba(preprocess_fn(image))
classes = classifier.class_list

print("Probabilities:")
for pred, cls in zip(prediction, classes):
  print(f"\t{cls}: {pred:.2%}")